# Project 3 - Richardson Final Final

In this project, you will perform a logistic regression on the admissions data we've been working with in projects 1 and 2.

In [42]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
import pylab as pl
import numpy as np
import os


In [43]:
df_raw = pd.read_csv("/Users/squirrel/Documents/project03/admissions.csv")
df = df_raw.dropna() 
df.head(10)

,admit,gre,gpa,prestige
0,0,380.0,3.61,3.0
1,1,660.0,3.67,3.0
2,1,800.0,4.00,1.0
3,1,640.0,3.19,4.0
4,0,520.0,2.93,4.0
5,1,760.0,3.00,2.0
6,1,560.0,2.98,1.0
7,0,400.0,3.08,2.0
8,1,540.0,3.39,3.0
9,0,700.0,3.92,2.0


In [44]:
df.tail(5)

,admit,gre,gpa,prestige
395,0,620.0,4.00,2.0
396,0,560.0,3.04,3.0
397,0,460.0,2.63,2.0
398,0,700.0,3.65,2.0
399,0,600.0,3.89,3.0


In [45]:
df.describe()

,admit,gre,gpa,prestige
count,397.000000,397.000000,397.000000,397.000000
mean,0.317380,587.858942,3.392242,2.488665
std,0.466044,115.717787,0.380208,0.947083
min,0.000000,220.000000,2.260000,1.000000
25%,0.000000,520.000000,3.130000,2.000000
50%,0.000000,580.000000,3.400000,2.000000
75%,1.000000,660.000000,3.670000,3.000000
max,1.000000,800.000000,4.000000,4.000000


## Part 1. Frequency Tables

#### 1. Let's create a frequency table of our variables

In [46]:
# frequency table for prestige and whether or not someone was admitted
#df.describe()
ct00 = pd.crosstab(index=df_raw["prestige"],  # Make a crosstab
                              columns="count") 

In [47]:
ct00

col_0,count
prestige,
1.0,61
2.0,150
3.0,121
4.0,67


## Part 2. Return of dummy variables

#### 2.1 Create class or dummy variables for prestige 

In [48]:
dummy_ranks = pd.get_dummies(df_raw['prestige'], prefix='prestige')

In [49]:
dummy_ranks.head(10)

,prestige_1.0,prestige_2.0,prestige_3.0,prestige_4.0
0,0,0,1,0
1,0,0,1,0
2,1,0,0,0
3,0,0,0,1
4,0,0,0,1
5,0,1,0,0
6,1,0,0,0
7,0,1,0,0
8,0,0,1,0
9,0,1,0,0


#### 2.2 When modeling our class variables, how many do we need? 



Answer: 3

## Part 3. Hand calculating odds ratios

Develop your intuition about expected outcomes by hand calculating odds ratios.

In [50]:
#crosstab prestige 1 admission 
# frequency table cutting prestige and whether or not someone was admitted
ct01 = handCalc.groupby('prestige_1.0').admit.count()

In [51]:
pd.crosstab(handCalc.admit,handCalc['prestige_1.0'])

prestige_1.0,0,1
admit,,
0,243,28
1,93,33


#### 3.1 Use the cross tab above to calculate the odds of being admitted to grad school if you attended a #1 ranked college

In [52]:
33/28

1.1785714285714286

In [53]:
#predictors=['prestige_1.0']
#lm = LogisticRegression()
#lm.fit(ct01[predictors], df['admit'])

#### 3.2 Now calculate the odds of admission if you did not attend a #1 ranked college

In [54]:
93/243

0.38271604938271603

#### 3.3 Calculate the odds ratio

In [55]:
(33/28)/(93/243)

3.079493087557604

#### 3.4 Write this finding in a sentenance: 

Answer: The odds of admittance given attendance at a Tier 1 school is 3.079 greater than those who did not attend a Tier 1 school.

#### 3.5 Print the cross tab for prestige_4

In [56]:
pd.crosstab(handCalc.admit,handCalc['prestige_4.0'])

prestige_4.0,0,1
admit,,
0,216,55
1,114,12


#### 3.6 Calculate the OR 

In [57]:
(12/55)/(114/216)

0.4133971291866028

#### 3.7 Write this finding in a sentence

Answer: The odds of being admitted from a Tier 4 school is less than half (.4134 OR) than someone who graduated from a Tier 1 school.

## Part 4. Analysis

In [58]:
df_raw = pd.read_csv("/Users/squirrel/Documents/project03/admissions.csv")
df = df_raw.dropna() 
df.head(10)

,admit,gre,gpa,prestige
0,0,380.0,3.61,3.0
1,1,660.0,3.67,3.0
2,1,800.0,4.00,1.0
3,1,640.0,3.19,4.0
4,0,520.0,2.93,4.0
5,1,760.0,3.00,2.0
6,1,560.0,2.98,1.0
7,0,400.0,3.08,2.0
8,1,540.0,3.39,3.0
9,0,700.0,3.92,2.0


In [59]:
# create a clean data frame for the regression
cols_to_keep = ['admit', 'gre', 'gpa']
data = df[cols_to_keep].join(dummy_ranks.ix[:, 'prestige_2.0':])
data.head()

/Users/squirrel/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  This is separate from the ipykernel package so we can avoid doing imports until


,admit,gre,gpa,prestige_2.0,prestige_3.0,prestige_4.0
0,0,380.0,3.61,0,1,0
1,1,660.0,3.67,0,1,0
2,1,800.0,4.00,0,0,0
3,1,640.0,3.19,0,0,1
4,0,520.0,2.93,0,0,1


We're going to add a constant term for our Logistic Regression. The statsmodels function we're going to be using requires that intercepts/constants are specified explicitly.

In [60]:
# manually add the intercept
data['intercept'] = 1.0      #'data' was defined in step [59] above

#### 4.1 Set the covariates to a variable called train_cols

In [62]:
train_cols = ['admit', 'gre', 'gpa','prestige_2.0','prestige_3.0','prestige_4.0']

In [63]:
train_cols

['admit', 'gre', 'gpa', 'prestige_2.0', 'prestige_3.0', 'prestige_4.0']

#### 4.2 Fit the model

In [64]:
logit = sm.Logit(data['admit'], data[train_cols])

In [65]:
result = logit.fit()

         Current function value: 0.000000
         Iterations: 35


/Users/squirrel/anaconda/lib/python3.6/site-packages/statsmodels/base/model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


#### 4.3 Print the summary results

In [66]:
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  admit   No. Observations:                  397
Model:                          Logit   Df Residuals:                      391
Method:                           MLE   Df Model:                            5
Date:                Thu, 24 Aug 2017   Pseudo R-squ.:                   1.000
Time:                        00:32:39   Log-Likelihood:            -1.6854e-05
converged:                      False   LL-Null:                       -248.08
                                        LLR p-value:                5.400e-105
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
admit           52.4310   8306.853      0.006      0.995   -1.62e+04    1.63e+04
gre           7.044e-05      2.608    2.7e-05      1.000      -5.111       5.111
gpa             -9.8699   2139.916     -0.005      0.996   -4204.029    4184.289
prestige_2.0     9.0335   5428.147      0.002      0.999   -1.06e+04    1.06e+04
prestige_3.0     5.5691   4129.126      0.001      0.999   -8087.368    8098.506
prestige_4.0     2.4068   4146.129      0.001      1.000   -8123.857    8128.671
================================================================================

Complete Separation: The results show that there iscomplete separation.
In this case the Maximum Likelihood Estimator does not exist and the parameters
are not identified.
"""

#### 4.4 Calculate the odds ratios of the coeffiencents and their 95% CI intervals

hint 1: np.exp(X)

hint 2: conf['OR'] = params
        
           conf.columns = ['2.5%', '97.5%', 'OR']

In [74]:
params = result.params
conf = result.conf_int()
conf['OR'] = params
conf.columns = ['2.5%', '97.5%', 'OR']
np.exp(conf)

/Users/squirrel/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:5: RuntimeWarning: overflow encountered in exp
  """


,2.5%,97.5%,OR
admit,0.00000,inf,5.895180e+22
gre,0.00603,165.862265,1.000070e+00
gpa,0.00000,inf,5.170985e-05
prestige_2.0,0.00000,inf,8.379418e+03
prestige_3.0,0.00000,inf,2.622020e+02
prestige_4.0,0.00000,inf,1.109820e+01


#### 4.5 Interpret the OR of Prestige_2

Answer: ?????????

#### 4.6 Interpret the OR of GPA

Answer: ????????

## Part 5: Predicted probablities


As a way of evaluating our classifier, we're going to recreate the dataset with every logical combination of input values. This will allow us to see how the predicted probability of admission increases/decreases across different variables. First we're going to generate the combinations using a helper function called cartesian (above).

We're going to use np.linspace to create a range of values for "gre" and "gpa". This creates a range of linearly spaced values from a specified min and maximum value--in our case just the min/max observed values.

In [ ]:
def cartesian(arrays, out=None):
    """
    Generate a cartesian product of input arrays.
    Parameters
    ----------
    arrays : list of array-like
        1-D arrays to form the cartesian product of.
    out : ndarray
        Array to place the cartesian product in.
    Returns
    -------
    out : ndarray
        2-D array of shape (M, len(arrays)) containing cartesian products
        formed of input arrays.
    Examples
    --------
    >>> cartesian(([1, 2, 3], [4, 5], [6, 7]))
    array([[1, 4, 6],
           [1, 4, 7],
           [1, 5, 6],
           [1, 5, 7],
           [2, 4, 6],
           [2, 4, 7],
           [2, 5, 6],
           [2, 5, 7],
           [3, 4, 6],
           [3, 4, 7],
           [3, 5, 6],
           [3, 5, 7]])
    """

    arrays = [np.asarray(x) for x in arrays]
    dtype = arrays[0].dtype

    n = np.prod([x.size for x in arrays])
    if out is None:
        out = np.zeros([n, len(arrays)], dtype=dtype)

    m = n / arrays[0].size
    out[:,0] = np.repeat(arrays[0], m)
    if arrays[1:]:
        cartesian(arrays[1:], out=out[0:m,1:])
        for j in xrange(1, arrays[0].size):
            out[j*m:(j+1)*m,1:] = out[0:m,1:]
    return out

In [ ]:
# instead of generating all possible values of GRE and GPA, we're going
# to use an evenly spaced range of 10 values from the min to the max 
gres = np.linspace(data['gre'].min(), data['gre'].max(), 10)
print gres
# array([ 220.        ,  284.44444444,  348.88888889,  413.33333333,
#         477.77777778,  542.22222222,  606.66666667,  671.11111111,
#         735.55555556,  800.        ])
gpas = np.linspace(data['gpa'].min(), data['gpa'].max(), 10)
print gpas
# array([ 2.26      ,  2.45333333,  2.64666667,  2.84      ,  3.03333333,
#         3.22666667,  3.42      ,  3.61333333,  3.80666667,  4.        ])


# enumerate all possibilities
combos = pd.DataFrame(cartesian([gres, gpas, [1, 2, 3, 4], [1.]]))

In [ ]:
#This makes zero sense to me

#### 5.1 Recreate the dummy variables

In [75]:
# recreate the dummy variables

# keep only what we need for making predictions

dummy_ranks = pd.get_dummies(df_raw['prestige'], prefix='prestige')

In [76]:
dummy_ranks

,prestige_1.0,prestige_2.0,prestige_3.0,prestige_4.0
0,0,0,1,0
1,0,0,1,0
2,1,0,0,0
3,0,0,0,1
4,0,0,0,1
5,0,1,0,0
6,1,0,0,0
7,0,1,0,0
8,0,0,1,0
9,0,1,0,0


#### 5.2 Make predictions on the enumerated dataset

?????????

#### 5.3 Interpret findings for the last 4 observations

Answer: ??????????

## Bonus

Plot the probability of being admitted into graduate school, stratified by GPA and GRE score.